In [17]:
import sys
sys.path.append("../")

from photutils.aperture import RectangularAperture, EllipticalAnnulus

import cubespa

from matplotlib import pyplot as plt

import numpy as np

from tqdm import tqdm

from astropy.io import fits

In [3]:
phangs_fn = "../personal/NGC4858_12m+7m_co21_pbcorr_crop.fits"
phangs_mms = "../personal/mm_out_phangs/NGC4858_PHANGS.3.0_1.9"

stellar_ra, stellar_dec = 194.75871125601466, 28.115690196322245


phangs = cubespa.CubeSPA(phangs_fn, mom_maps=phangs_mms, limits="auto", center=(stellar_ra, stellar_dec, "radec"))


In [4]:
def quadrants(q1_angle, offset = 90):
    return [q1_angle + i * 90 + offset for i in range(4)]

def halfline_points(cent, length, angle):

    p1 = (cent[0] , cent[0] + length * np.cos(angle))
    p2 = (cent[1], cent[1] + length * np.sin(angle))
    
    return (p1, p2)


def line_points(cent, length, angle):
    p1 = (cent[0] - length * np.cos(angle), cent[0] + length * np.cos(angle))
    p2 = (cent[1] - length * np.sin(angle), cent[1] + length * np.sin(angle))

    
    return (p1, p2)


def sample_ellipse(x, y, a, e, theta, pa, num_points=100):

    # Calculate semiminor axis
    b = a * (1 - e)

    # Parametric equation for the ellipse
    x_coords = x + a * np.cos(theta) * np.cos(pa) - b * np.sin(theta) * np.sin(pa)
    y_coords = y + a * np.cos(theta) * np.sin(pa) + b * np.sin(theta) * np.cos(pa)

    # Return the sampled coordinates as a NumPy array
    return x_coords, y_coords


def beam_ellipse(x, y, a, b, pa, num_points=100):

    theta = np.linspace(0, np.pi * 2, 50)
    # Calculate semiminor axis

    # Parametric equation for the ellipse
    x_coords = x + a * np.cos(theta) * np.cos(pa) - b * np.sin(theta) * np.sin(pa)
    y_coords = y + a * np.cos(theta) * np.sin(pa) + b * np.sin(theta) * np.cos(pa)

    # Return the sampled coordinates as a NumPy array
    return x_coords, y_coords


In [19]:
thetas = (np.linspace(0, 2 * np.pi, 90)) # These thetas are CCW from +ve y-axis
pa_deg, inc_deg = 30, 38
pa, inc =  np.deg2rad(pa_deg + 90), np.deg2rad(inc_deg)

bar_pa = 65

bmaj, bmin, bpa = phangs.beam

data_co = phangs.mom_maps.mom0.data
stellar_x, stellar_y = phangs.center


In [11]:
thetas = (np.linspace(0, 2 * np.pi, 90)) # These thetas are CCW from +ve y-axis
# thetas = np.linspace(0, np.pi, 20)

dist, length = 70, 20
width=10


data_nonans = np.copy(phangs.mom_maps.mom0.data)
data_nonans[np.isnan(phangs.mom_maps.mom0.data)] = 0


# plt.xlim(200, 550)
# plt.ylim(400, 900)

fluxes_outer = []
mean_fluxes_outer = []
flux_stds_outer = []

for i, theta in enumerate(thetas):
    theta_adj = theta # + np.pi/2 # This theta is CCW from +ve x-axis
    px, py = phangs.center[0] + dist * np.cos(theta_adj), phangs.center[1] + dist  * np.sin(theta_adj)

    rect = RectangularAperture((px, py), width, length, theta=theta_adj + np.pi / 2)

    mask = rect.to_mask(method='center').to_image(shape=data_co.shape)
    data_slice = np.copy(data_co)
    data_slice[mask == 0] = np.nan

    fluxes_outer.append(np.nansum(data_slice))
    mean_fluxes_outer.append(np.nanmean(data_slice))
    flux_stds_outer.append(np.nanstd(data_slice))



<ipython-input-11-5ae339303439>:30: RuntimeWarning: Mean of empty slice
  mean_fluxes_outer.append(np.nanmean(data_slice))
/Users/hsouch/anaconda3/envs/jelly/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


In [12]:
def theta_range(shape, x0, y0, theta, dtheta):
    ys, xs = np.mgrid[:shape[0], :shape[1]].astype(float)
    xs -= x0
    ys -= y0
    
    thetas = (np.arctan2(ys, xs) ) % (2 * np.pi)

    theta_mask = np.ones(thetas.shape)

    
    theta_min = (theta - dtheta / 2) % (2 * np.pi)
    theta_max = (theta + dtheta / 2) % (2 * np.pi)
    
    # Adjust for cases where theta_max < theta_min due to wrapping around 2pi
    if theta_max < theta_min:
        # In this case, we need to handle the wrap around.
        theta_mask = np.logical_or(thetas >= theta_min, thetas <= theta_max)
    else:
        # Regular case where the range does not wrap around
        theta_mask = np.logical_and(thetas >= theta_min, thetas <= theta_max)

    return theta_mask.astype(int)


theta_range(phangs.mom_maps.mom0.data.shape, phangs.center[0], phangs.center[1], np.pi / 4, 4 * np. pi / 180)

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [46]:
def segment_sum(data, radius, pa, angle_min, annulus_width=50):
    #ell_annulus = EllipticalAperture((stellar_x, stellar_y), radius, (radius * (1-eps)), theta = pa)
    ell_annulus = EllipticalAnnulus((stellar_x, stellar_y), radius, radius + annulus_width, 
                                    (radius + annulus_width) * (1-eps), theta = pa)
    
    
    mask = ell_annulus.to_mask(method='center').to_image(shape=data_co.shape)
    data_init_mask = np.copy(data)
    data_init_mask[mask == 0] = np.nan
    
    
    theta_mask = theta_range(data.shape, stellar_x, stellar_y, angle_min - np.pi / 4, np.pi / 2)
    
    data_init_mask[theta_mask == 0] = np.nan
    
    
    return np.nansum(data_init_mask)
    
    
qs = np.deg2rad(quadrants(75, offset=0))

eps = 0.21
annulus_width = 30
radii = [10, 40, 70, 100, 130, 160]

quadrant_fluxsums = []
for r in radii:
    row_sums = []
    for q in qs:
        row_sums.append(segment_sum(data_co, r, pa, q, annulus_width=200))
    quadrant_fluxsums.append(row_sums)


quadrant_fluxsums = np.asarray(quadrant_fluxsums)

print(quadrant_fluxsums)

[[808.52844   430.74167   412.2507    478.9904   ]
 [386.63745    19.881962    1.063072    1.4727652]
 [247.37187    15.576887    0.          0.       ]
 [148.76698    15.1991825   0.          0.       ]
 [ 31.036644    5.514929    0.          0.       ]
 [  0.          0.          0.          0.       ]]


In [39]:
eps = 0.21

radius = 120

def process_azimuth(data, radius, pa, eps, thetas, annulus_width=50, trim_nans=False):

    
    ell_annulus = EllipticalAnnulus((stellar_x, stellar_y), radius, radius + annulus_width, 
                                    (radius + annulus_width) * (1-eps), theta = pa)
    mask = ell_annulus.to_mask(method='center').to_image(shape=data_co.shape)
    
    data_init_mask = np.copy(data)
    data_init_mask[mask == 0] = np.nan

    fluxes, flux_stds = [], []

    for i, theta in tqdm(enumerate(thetas[:])):
    
        theta_mask = theta_range(data_co.shape, stellar_x, stellar_y, theta, 5 * np.pi / 180)
    
        this_data = np.copy(data_init_mask)
        this_data[theta_mask == 0] = np.nan
        
        fluxes.append(np.nanmedian(this_data))
        flux_stds.append(np.nanstd(this_data))

    if trim_nans:
        fluxes = np.array(fluxes)
        fluxes[np.isnan(fluxes)] = 0

        flux_stds = np.array(flux_stds)
        flux_stds[np.isnan(flux_stds)] = 0
    
    return (np.array(fluxes), np.array(flux_stds), ell_annulus)


sets = []
radii = [40, 70, 100, 130]
for r in radii:
    sets.append(process_azimuth(data_co, r, pa, eps, thetas, annulus_width=28, trim_nans=True))

0it [00:00, ?it/s]<ipython-input-39-22d6b00cd471>:24: RuntimeWarning: All-NaN slice encountered
  fluxes.append(np.nanmedian(this_data))
/Users/hsouch/anaconda3/envs/jelly/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
90it [00:00, 269.46it/s]
90it [00:00, 275.18it/s]
90it [00:00, 291.53it/s]
90it [00:00, 278.87it/s]


In [45]:
def azimuthal_plot(data, sets, out_filename=f"Azimuthal_plot_flux.pdf", quadrant_sums=None,
                   ylabel=r'Mean I [Jy km/s]', xlabel=r'$\phi$ [CCW from X-Axis]',
                  plot_fiducial=True, ylim=None,
                  log_img = False, vmin=-50, vmax=50, cmap="rainbow"):

    fig = plt.figure(figsize=(17, 8), facecolor="white")
    thetas_adj = np.rad2deg(thetas)
    
    xwidth = 0.50
    yheight = 0.90 / len(sets)
    yspacing = 0
    ymin_pad = 0.060
    
    ylim = [-0.05, 0.9] if ylim is None else ylim
    
    axes = []
    for i in range(len(sets) -1, -1, -1):
        axes.append(fig.add_axes([0.05, ymin_pad + i * (yheight + yspacing), xwidth, yheight]))
    
    annuli_colors = ["blue", "green", "red", "orange", "cyan", "magenta", "lime", "crimson"]
    
    img_ax = fig.add_axes([0.5, 0.01, 0.48, 0.98])
    
    for i, set in enumerate(sets):

        ymin, ymax = np.min(set[0]), np.max(set[0])
        dy = ymax - ymin
        if dy > 1e-5:
            axes[i].set_ylim(ymin - dy/5, ymax + dy/5)


        # axes[i].scatter(thetas_adj, set[0], cmap=cmap, c = thetas_adj / np.max(thetas_adj))
        axes[i].plot(thetas_adj, set[0], color=annuli_colors[i], zorder=3)
        axes[i].fill_between(thetas_adj, set[0] - set[1], set[0] + set[1], color=annuli_colors[i], zorder=2, alpha=0.3)
        axes[i].fill_between(thetas_adj, set[0] - 3 * set[1], set[0] + 3 * set[1], color=annuli_colors[i], zorder=2, alpha=0.1)
    
        axes[i].axhline(0, ls="dashed", color="Grey", zorder=2, alpha=0.5)
            
        # Plot fiducial velocities
        if plot_fiducial and i != 0:
            axes[i].plot(thetas_adj, sets[0][0], color="Grey", zorder=2, alpha=0.5)
    
        if i is not len(set):
            axes[i].tick_params(labelbottom=False)    
            

    qs = quadrants(75, offset=0)

    
    styles = ['solid', 'dashed', 'dashdot', 'dotted']
    quadrant_labels = ['Trailing Side\nCompression Weak', "\tRP Stronger", 
                       'Leading Side\nCompression Strong', '        RP Weaker']
    
    for i, ax in enumerate(axes):
        
        ylims = ax.get_ylim()

        for j, q in enumerate(qs):
            ax.axvline(q, color="black", linestyle=styles[j], lw=3)
            
            text_x = np.max([q - 80, 10])
            ax.text(text_x, ylims[1] * 0.80, f'Q{j + 1}')
            
            if quadrant_sums is not None:
                qsum = np.round(quadrant_sums[i, j], 2)
                ax.text(text_x, ylims[1] * 0.65, r'$S_{CO} =$' + str(qsum))

        
                   
        ax.axvline(bar_pa + 90, color="Grey")
        ax.axvline(bar_pa + 90 + 180, color="Grey")
    
        ax.set_ylabel(ylabel, fontsize=10)
    
    axes[-1].set_xlabel(xlabel)

    # Add and clean up the image plot
    if log_img:
        img_map = img_ax.imshow(np.log10(data), origin="lower", cmap=cmap, vmin=vmin, vmax=vmax)
    else:
        img_map = img_ax.imshow(data, origin="lower", cmap=cmap, vmin=vmin, vmax=vmax)
    
    img_xmin, img_xmax, img_ymin, img_ymax = phangs.limits
    img_ax.set_xlim(img_xmin, img_xmax)
    img_ax.set_ylim(img_ymin, img_ymax)

    x_shifts = [-20, -5, 20, 5]
    y_shifts = [0, -20, 0, 10]
    
    for i, q in enumerate(qs):
        ylims = axes[0].get_ylim()
        
        if quadrant_sums is not None:
            qsum = np.sum(np.transpose(quadrant_sums)[i])
            qsum = np.round(qsum, 2)
            
            axes[0].text(q - 80, ylims[1] * 1.1, r'$S_{CO}=$' + str(qsum))
    
        q = np.deg2rad(q)
        points = halfline_points((stellar_x, stellar_y), 100, q)
        img_ax.plot(points[0], points[1], color="black", lw=5, linestyle=styles[i])

        mean_x, mean_y = np.mean(points[0]), np.mean(points[1])
        print(mean_x, mean_y)
        
        img_ax.text(mean_x + x_shifts[i], mean_y + y_shifts[i], 
                    f"Q{(i + 1) % 4 + 1}",  fontsize=20, backgroundcolor="white")
        
    img_ax.set_xticks([])
    img_ax.set_yticks([])
    
    bar_points = line_points((stellar_x, stellar_y), 50, np.deg2rad(bar_pa + 90))
    img_ax.plot(bar_points[0], bar_points[1], color="white", ls="solid", lw=3)
    plt.colorbar(mappable=img_map, ax=img_ax, fraction=0.06, pad=0.02)
    
    for i, s in enumerate(sets):
        s[2].plot(color=annuli_colors[i], lw=3)
    
    # Draw beam
    beam_x, beam_y = beam_ellipse(220, 240, bmaj /2, bmin /2, np.deg2rad(bpa))
    img_ax.plot(beam_x, beam_y, color="black", zorder=2)
        
    plt.savefig(out_filename, dpi=200)
    plt.savefig(f"{out_filename.removesuffix('.pdf')}.png", dpi=200)
    plt.close()


azimuthal_plot(data_co, sets=sets, out_filename=f"../personal/flux_analysis/Azimuthal_plot_flux.pdf", 
               plot_fiducial=False, ylim=[-0.05, 0.3],
               log_img=True, vmin=-5, vmax=1, cmap="Greys", quadrant_sums=quadrant_fluxsums)

134.98817800767128 155.48784422591578
73.75093443809183 120.13250516658842
109.10627349741921 58.895261597008954
170.34351706699866 94.25060065633633
